# PET-проект: Анализ A/B теста дохода пользователей

В этом проекте мы анализируем результаты A/B теста на сайте. Основная цель — проверить гипотезу о том, что новый вариант оформления (`variant`) увеличивает доход пользователей (`revenue`) по сравнению с контрольной группой (`control`).

**Импортируем необходимые библиотеки:**
- `pandas` и `numpy` — работа с данными и вычисления
- `matplotlib` и `seaborn` — визуализация

In [ ]:
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
import seaborn as sns

## Шаг 1: Загрузка и первичный осмотр данных

Подгружаем CSV-файл с результатами эксперимента и проверяем его структуру:

- `user_id` — идентификатор пользователя
- `VARIANT_NAME` — тип группы (control / variant)
- `REVENUE` — доход, принесённый пользователем

Выводим размер датафрейма и первые строки для ознакомления.

In [ ]:
url = "https://raw.githubusercontent.com/ilya-lysenko-An/Pet-A.B/main/AB_Test_Results.csv"
df = pd.read_csv(url)
print("Проверка дата фрейма", df.shape)
print("Метрики дата фрейма:")
print(df.head().T)

## Шаг 2: Распределение пользователей по группам

Считаем:
- количество пользователей в каждой группе
- процентное распределение

Это необходимо, чтобы убедиться, что группы примерно равны и эксперимент корректно сбалансирован.


In [ ]:
group_count = df['VARIANT_NAME'].value_counts()
print("Колличество пользователей в каждой группе")
print(group_count)
group_percent = df['VARIANT_NAME'].value_counts(normalize=True) * 100
print("\nПроцентное распределение")
print(group_percent)

Визуализируем распределение пользователей по группам с помощью `countplot`.  
График позволяет наглядно проверить баланс между control и variant.


In [ ]:
plt.figure(figsize=(6,4))
sns.countplot(data = df, x = 'VARIANT_NAME')
plt.title("Колличество пользователей по группам")
plt.xlabel("Группа")
plt.ylabel("Колличество пользователей")
plt.show()

## Шаг 3: Описательная статистика дохода

Считаем базовые метрики дохода по группам:
- среднее (mean)
- стандартное отклонение (std)
- медиана и квартели
- минимальные и максимальные значения

Это помогает понять распределение дохода, наличие выбросов и скошенность данных.


In [ ]:
group_stats = df.groupby("VARIANT_NAME")['REVENUE'].describe()
print("Описательная статистика дохода по группам")
print(group_stats)

## Шаг 4: Bootstrap-анализ разницы средних

Используем метод bootstrap для оценки разницы средних доходов между группами:

1. Берём случайные выборки с возвращением из каждой группы
2. Считаем среднее для каждой выборки
3. Вычисляем разницу средних (variant - control)
4. Формируем распределение разницы средних
5. Строим 95% доверительный интервал
6. Рассчитываем uplift — практический эффект в процентах

Bootstrap позволяет оценить статистическую значимость разницы без предположений о нормальном распределении.


In [ ]:
control = df[df['VARIANT_NAME'] == 'control']['REVENUE'].values
variant = df[df['VARIANT_NAME'] == 'variant']['REVENUE'].values

n_iterations = 1000
rng = np.random.default_rng(seed=42)
diff_means = []

for _ in range(n_iterations):
    sample_control = rng.choice(control, size = len(control), replace = True)
    sample_variant = rng.choice(variant, size = len(variant), replace = True)
    diff = sample_variant.mean() - sample_control.mean()
    diff_means.append(diff)

diff_means = np.array(diff_means)

ci_lower = np.percentile(diff_means, 2.5)
ci_upper = np.percentile(diff_means, 97.5)
uplift = (variant.mean() - control.mean()) / control.mean() * 100

print(f"Средний доход control: {control.mean():.4f}")
print(f"Средний доход variant: {variant.mean():.4f}")
print(f"Разница средних (variant - control): {variant.mean() - control.mean():.4f}")
print(f"95% доверительный интервал разницы: [{ci_lower:.4f}, {ci_upper:.4f}]")
print(f"Uplift (в процентах): {uplift:.2f}%")

## Шаг 5: Визуализация распределения разницы средних

Строим гистограмму распределения разницы средних, полученной методом bootstrap.  

- Красная пунктирная линия соответствует нулевой разнице (нет эффекта)
- Если большая часть распределения находится слева/справа от нуля, можно делать вывод о тенденции изменения дохода

Этот график служит наглядным доказательством результата эксперимента.


In [ ]:
plt.figure(figsize=(8,4))
plt.hist(diff_means, bins=50, color='skyblue', edgecolor='black')
plt.axvline(0, color='red', linestyle='--', label='No difference')
plt.title("Bootstrap распределение разницы средних (variant - control)")
plt.xlabel("Разница средних")
plt.ylabel("Количество повторов")
plt.legend()
plt.show()

## Шаг 6: Оценка статистической значимости

Для проверки гипотезы используем bootstrap-распределение разницы средних доходов.

### Формулировка гипотез:
- **H₀ (нулевая гипотеза):** средний доход в группе `variant` не превышает доход в группе `control`
- **H₁ (альтернативная гипотеза):** новый вариант (`variant`) увеличивает доход пользователей

Так как гипотеза изначально направленная (ожидаем рост дохода), используем **односторонний p-value**.

In [ ]:
p_value_one_side = np.mean(diff_means >= 0)
p_value_two_side = np.mean(np.abs(diff_means) >= abs(control.mean() - variant.mean()))
print(f"P-value односторонний (variant > control): {p_value_one_side:.4f}")
print(f"P-value двусторонний (variant ≠ control): {p_value_two_side:.4f}")

## Итоговые выводы

- Эксперимент корректно проведён: группы сбалансированы по числу пользователей
- Распределение дохода сильно скошено и содержит выбросы
- Метод bootstrap показал отрицательную разницу средних доходов
- 95% доверительный интервал включает ноль
- p-value указывает на отсутствие статистически значимого эффекта роста дохода

### Рекомендации:
- Не внедрять новый вариант оформления в текущем виде
- Провести дополнительный анализ причин падения дохода
- Возможные следующие шаги:
  - сегментация пользователей
  - анализ конверсии отдельно от дохода
  - запуск нового A/B теста с доработанным дизайном